In [406]:
from src.models.subreddit_analysis import SubredditAnalysis
from imports import *
import plotly.express as px
from nltk.probability import FreqDist


li = ['posts', 'comments', 'results']

In [407]:
pd.set_option('display.max_colwidth', 1000)

# 1. Home - Conflicting Viewpoints in Reddit Submissions

**Welcome to BEReddiT**

***Subreddit Search Bar***  
***Subreddit Submission Bar***

*Brief Subreddit Metadata* [TODO]

- Num of moderators
- Num of followers and members
- Flairs
- Number of posts and comments for analysis

*Basic Subreddit Data*

- Posts
    - Total Number of Posts
- Comments
    - Total Number of Comments

### Number of posts and comments retrieved after preprocessing raw data.

In [408]:
sub = ['antiwork', 'computerscience', 'loseit', 'WorkReform', 'movies', 'geopolitics', 'AmItheAsshole'] #dropdown

path = '../data/results/'+sub[3]+'_hot_'

posts_df = pd.read_csv(path+li[0]+'.csv')
comments_df = pd.read_csv(path+li[1]+'.csv')
res_df = pd.read_csv(path+li[2]+'.csv')

print('posts: ',posts_df.shape)
print('comments: ', comments_df.shape)
print('sub-analysis results: ', res_df.shape)

posts:  (160, 16)
comments:  (10056, 19)
sub-analysis results:  (3612, 35)


In [409]:
posts_df.head()

index post_id  \
0      1  t23u79   
1      3  t1vmtz   
2     12  t23eep   
3     15  t1logh   
4     20  t26dou   

                                                                                                                                                                                                                                                                                                        title  \
0                                                                                                                                                                                                                                                                                 Wtf is wrong with the world   
1                                                                                                                                                                                                                                                                  New CEO and HR at company made big changes   
2                                                                                                                                                                                                           I think we should all collectively stop applying to jobs that don't have the salary range posted.   
3                                                                                                                                                            There needs to be a term for the practice of intentionally understaffing to increase profits at the cost of the employees' well being. Thoughts?   
4  New Mexico passed a bill to increase teacher salaries by setting 3 salary tiers across the state. Tier 1: 1st year teachers will make a minimum of $50,000. Tier 2: teachers with 3-5 years of experience will make a minimum of $60,000. Tier 3: more experienced teacher will make a minimum of $70,000.   

  flair  score  upvote_ratio   subreddit  \
0   NaN   1286          0.95  WorkReform   
1   NaN   1510          0.99  WorkReform   
2   NaN    129          0.98  WorkReform   
3   NaN   1009          0.99  WorkReform   
4   NaN     16          0.94  WorkReform   

                                                                                                     url  \
0                       https://www.reddit.com/r/WorkReform/comments/t23u79/wtf_is_wrong_with_the_world/   
1        https://www.reddit.com/r/WorkReform/comments/t1vmtz/new_ceo_and_hr_at_company_made_big_changes/   
2  https://www.reddit.com/r/WorkReform/comments/t23eep/i_think_we_should_all_collectively_stop_applying/   
3      https://www.reddit.com/r/WorkReform/comments/t1logh/there_needs_to_be_a_term_for_the_practice_of/   
4      https://www.reddit.com/r/WorkReform/comments/t26dou/new_mexico_passed_a_bill_to_increase_teacher/   

   num_comments  \
0           261   
1           102   
2            13   
3           182   
4             8   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      body  \
0               

In [410]:
comments_df.head()

index post_id comment_id  parent_id  \
0      0  t1yqe8    hyir8lg  t3_t1yqe8   
1      1  t1yqe8    hyj154m  t3_t1yqe8   
2      2  t1yqe8    hyiu2zu  t3_t1yqe8   
3      3  t1yqe8    hyj1zum  t3_t1yqe8   
4      4  t1yqe8    hyiypwh  t3_t1yqe8   

                                                                                                                                                           comment  \
0  as much as it is a ‚Äúno duh‚Äù moment, yes, they need to have official studies and data so it can be pointed to and maybe fixed one day. get it down in the record   
1                              pandemic aid?  you mean when the old people in congress gave us like $14 and expected it to last a year and a half like it is 1905?   
2                                                            ‚Äúinflation‚Äù.  thanks to the internet, we know this is just record profits, greed and shady ass ceo‚Äôs.   
3                    yup. getting laid off, inflation, and the insane rise of cost of homes have pretty much killed my dream i already left everything behind for.   
4                                             for the delusional people that think amazon lost money and unemployed people are living la dolce vida on food stamps   

   up_vote_count  down_vote_count  controversiality  total_awards_received  \
0            386                0                 0                      1   
1             86                0                 0                      0   
2            265                0                 0                      0   
3             32                0                 0                      0   
4             21                0                 0                      0   

   score  is_locked  is_collapsed  is_submitter   created_utc  \
0    386      False         False         False  1.645888e+09   
1     86      False         False         False  1.645892e+09   
2    265      False         False         False  1.645889e+09   
3     32      False         False         False  1.645893e+09   
4     21      False         False         False  1.645891e+09   

                                                                                                                                                        comment_word_token  \
0                   ['much', '‚Äú', 'duh', '‚Äù', 'moment', ',', 'yes', ',', 'need', 'official', 'study', 'data', 'point', 'maybe', 'fix', 'one', 'day', '.', 'get', 'record']   
1                     ['pandemic', 'aid', '?', 'mean', 'old', 'people', 'congress', 'give', 'u', 'like', '$', '14', 'expect', 'last', 'year', 'half', 'like', '1905', '?']   
2                                        ['‚Äú', 'inflation', '‚Äù', '.', 'thanks', 'internet', ',', 'know', 'record', 'profit', ',', 'greed', 'shady', 'as', 'ceo', '‚Äô', '.']   
3  ['yup', '.', 'get', 'laid', ',', 'inflation', ',', 'insane', 'rise', 'cost', 'home', 'pretty', 'much', 'kill', 'dream', 'already', 'left', 'everything', 'behind', '.']   
4                                   ['delusional', 'people', 'think', 'amazon', 'lose', 'money', 'unemployed', 'people', 'living', 'la', 'dolce', 'vida', 'food', 'stamp']   

                                                                                                                                                                                                                                                                                                                                                                             comment_tag  \
0                 [[('much', 'JJ')], [('‚Äú', 'NN')], [('duh', 'NN')], [('‚Äù', 'NN')], [('moment', 'NN')], [(',', ',')], [('yes', 'NNS')], [(',', ',')], [('need', 'NN')], [('official', 'NN')], [('studies', 'NNS')], [('data', 'NNS')], [('pointed', 'VBN')], [('maybe', 'RB')], [('fixed', 'VBN')], [('one', 'CD')], [('day', 'NN')], [('.', '.')], [('get', 'VB')], [('record', 'NN')]]   
1                                 [[('pandemic', 'NN')], [('aid'

In [411]:
posts_df.columns, comments_df.columns, res_df.columns

(Index(['index', 'post_id', 'title', 'flair', 'score', 'upvote_ratio',
        'subreddit', ' url', 'num_comments', 'body', 'created',
        'body_word_token', 'body_tag', 'body_string', 'topics', 'sentiment'],
       dtype='object'),
 Index(['index', 'post_id', 'comment_id', 'parent_id', 'comment',
        'up_vote_count', 'down_vote_count', 'controversiality',
        'total_awards_received', 'score', 'is_locked', 'is_collapsed',
        'is_submitter', 'created_utc', 'comment_word_token', 'comment_tag',
        'body_string', 'topics', 'sentiment'],
       dtype='object'),
 Index(['post_index', 'post_id', 'post_title', 'flair', 'post_score',
        'post_upvote_ratio', 'subreddit', 'post_url', 'num_comments',
        'post_body', 'post_created', 'post_body_word_token', 'post_body_tag',
        'post_body_string', 'post_topics', 'post_sentiment', 'comment_index',
        'comment_id', 'parent_id', 'comment', 'comment_up_vote_count',
        'comment_down_vote_count', 'comment_cont

### Histograms of Score Distribution (Number of upvotes remaining) 

In [412]:
posts_score_hist = px.histogram(posts_df, 
                                x="score", 
                                title='Frequency Distribution of Post Score',
                                labels={'score':'Score', 'count':'Number of Posts'},
                                opacity=0.8,
                                color_discrete_sequence=['indianred'],
                                text_auto=True).update_layout(
                                yaxis_title="Number of Posts")
posts_score_hist.show()

In [477]:
comms_score_hist = px.histogram(comments_df,                                
                                x="score",
                                log_y=True,
                                title='Frequency Distribution of Comment Score',
                                opacity=0.8).update_layout(
                                xaxis_title="Score", yaxis_title="Number of Comments (log scale)")
comms_score_hist.update_xaxes(range=[0, 2000])
comms_score_hist.show()
# notify on app that more can be seen because graph was resized to observe data better.

### Submission Word Counts

In [414]:
comments_df.comment.astype(str)

0                         as much as it is a ‚Äúno duh‚Äù moment, yes, they need to have official studies and data so it can be pointed to and maybe fixed one day. get it down in the record
1                                                     pandemic aid?  you mean when the old people in congress gave us like $14 and expected it to last a year and a half like it is 1905?
2                                                                                   ‚Äúinflation‚Äù.  thanks to the internet, we know this is just record profits, greed and shady ass ceo‚Äôs.
3                                           yup. getting laid off, inflation, and the insane rise of cost of homes have pretty much killed my dream i already left everything behind for.
4                                                                    for the delusional people that think amazon lost money and unemployed people are living la dolce vida on food stamps
                                                            

In [415]:
posts_df['word_counts'] = posts_df.title.str.cat(posts_df.body, sep=" ").str.split().apply(len)
comments_df['word_counts'] = comments_df.comment.astype(str).str.split().apply(len)

In [416]:
fig = px.histogram(posts_df, 
                    x="word_counts",
                    title='Word-Count Distribution for Posts',
                    labels={'word_counts':'Word Count', 'count':'Number of Posts'},
                    opacity=0.8,
                    color_discrete_sequence=['indianred'],
                    text_auto=True).update_layout(
                    yaxis_title="Number of Posts")
fig.show()

In [476]:
fig = px.histogram(comments_df, 
                   x="word_counts",
                   log_y=True,
                    title='Word-Count Distribution for Comments',
                    opacity=0.8).update_layout(
                    xaxis_title="Word Count", yaxis_title="Number of Comments (log scale)")
comms_score_hist.update_xaxes(range=[0, 2000])
fig.show()

# 2. Relevance - Are the comments in discussions relevant to the submission?

In [418]:
res_df.comment_relevance

0       0.767590
1       0.352402
2       0.608428
3       0.455626
4       0.822468
          ...   
3607    0.327015
3608    0.779539
3609    0.737502
3610    0.656965
3611    0.416587
Name: comment_relevance, Length: 3612, dtype: float64

### Frequency Distribution of Overall Comment Relevance

In [443]:
hist_df = res_df.copy()
hist_df["color"] = np.select(
    [hist_df["comment_relevance"].gt(0.65), hist_df["comment_relevance"].lt(0.65)],
    ["green", "red"],
    "orange"
)
fig = px.histogram(hist_df, x="comment_relevance",
                    title='Distribution of Comment Relevance',
                    labels={'comment_relevance':'Comment Relevance Score', 'count':'Number of Comments'},
                    opacity=0.6, 
                    color="color",
                    color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "orange": "orange"
                    })

fig.update_layout(yaxis_title="Number of Comments", showlegend=False)

fig.add_vline(x=0.65, line_width=3, line_dash="dash", line_color="black")

#include lines for classifying relevant or not : t-test for relevance 0.65< 
fig.show()

In [420]:
import scipy.stats as stats

#perform one sample t-test
stats.ttest_1samp(a=res_df.comment_relevance, popmean=0.60, alternative='greater')

Ttest_1sampResult(statistic=-20.530498839492473, pvalue=1.0)

### p-value shows that this sub doesn't significantly better average relevance among its comments

In [421]:
relevance_mean = np.mean(res_df.comment_relevance)
relevance_mean

0.5227787155449929

### Show Comments after selecting post and Check Relevance in tabular form

In [447]:
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)
prods = list(res_df.head(100).comment)
vals = list(res_df.head(100).comment_relevance)


trace = dict(type = 'table',
             columnwidth= [200] + [100],
             header = dict(height = 50,
                           values = [['<b>Comment</b>'], ['<b>Relevance</b>']],
                           line = dict(color='rgb(50, 50, 50)'),
                           align = 'left', 
                           font = dict(color=['rgb(45, 45, 45)'], size=14),
                           fill = dict( color = 'rgb(235, 235, 235)' )
                              ),
             cells = dict(values = [prods, vals],
                          line = dict(color='#506784'),
                          align = 'left',
                          font = dict(color=['rgb(40, 40, 40)'], size=12),
                          format = [None, ",.2f"],
                          height = 30,
                          fill = dict(color=['rgb(245, 245, 245)',#unique color for the first column
                                            ['rgba(0, 250, 0, 0.8)' if val >= 0.65 else 'rgba(250, 0, 0, 0.8)' for val in vals] ]
                          #the cells in the second column are colored with green or red, according to their values(+ or -)
                                         )
                             )
                  )

In [448]:
layout = dict(width=1000, autosize=True, 
              title='Cells colored according to their values', showlegend=False)
fig= dict(data=[trace], layout=layout)
iplot(fig)

### Relevance score for a post is the number of relevant comments to a post out of the 100 sampled comments.

In [441]:
posts_df['post_relevance'] = list(res_df.groupby(["post_id"])['comment_relevance'].mean())
posts_df.post_relevance

0      0.543063
1      0.582970
2      0.499646
3      0.648297
4      0.658684
         ...   
155    0.510047
156    0.604501
157    0.559545
158    0.000000
159    0.381669
Name: post_relevance, Length: 160, dtype: float64

### Distribution of relevance scores: the following shows a distribution of how discussion across posts in this subreddit tends to be relevant. (A simpler form of showing overall comment relevance)

In [444]:
hist_df = posts_df.copy()
hist_df["color"] = np.select(
    [hist_df["post_relevance"].gt(0.65), hist_df["post_relevance"].lt(0.65)],
    ["green", "red"],
    "orange"
)
fig = px.histogram(hist_df, x="post_relevance",
                    title='Distribution of Post Relevance',
                    labels={'post_relevance':'Post Relevance Score', 'count':'Number of Posts'},
                    opacity=0.6, 
                    color="color",
                    color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "orange": "orange"
                    })

fig.update_layout(yaxis_title="Number of Posts", showlegend=False)

fig.add_vline(x=0.65, line_width=3, line_dash="dash", line_color="black")

#include lines for classifying relevant or not : t-test for relevance 0.65< 
fig.show()

# 3. Topic Modeling - What are posts talking about?

In [449]:
pd.options.plotting.backend = "plotly"

In [450]:
res_df.comment_topics

0                   -1_work_job_get_company
1                   -1_work_job_get_company
2                   -1_work_job_get_company
3                 11_frozen_veggie_bean_eat
4                   -1_work_job_get_company
                       ...                 
3607    5_boomer_generation_gen_millennials
3608         67_loan_student_predatory_debt
3609                -1_work_job_get_company
3610                -1_work_job_get_company
3611    5_boomer_generation_gen_millennials
Name: comment_topics, Length: 3612, dtype: object

In [471]:
fig = res_df.comment_topics.value_counts()[1:16].plot(kind="bar", title='<b>Top 15</b> Important Topics For Comments',
                    text_auto=True).update_layout(
                    xaxis_title="Topics", yaxis_title="Number of Comments")
fig.update_layout(showlegend=False)
fig.show()

In [452]:
posts_df.topics.value_counts()

-1_work_job_time_get            67
0_work_day_would_get            46
1_people_work_make_go           28
2_job_get_interview_position    19
Name: topics, dtype: int64

In [474]:
fig = posts_df.topics.value_counts()[1:].plot(kind="bar", title='Important Topics For Posts',
                    text_auto=True, color_discrete_sequence=['indianred']).update_layout(
                    xaxis_title="Topics", yaxis_title="Number of Posts")
fig.update_layout(showlegend=False)
fig.show()

# 4. Sentiment Analysis - Can we identify conflict with Sentiment?

In [503]:
hist_df = posts_df.copy()
hist_df["color"] = np.select(
    [hist_df["sentiment"].eq(5), hist_df["sentiment"].eq(1)],
    ["green", "red"],
    "blue"
)

fig = px.histogram(hist_df, 
                   x="sentiment",
                   title='Distribution of Sentiment for Posts',text_auto=True, 
                   color="color",
                   color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "blue": "blue"
                    }).update_layout(
                   xaxis_title="Sentiment", yaxis_title="Number of Posts", showlegend=False)
fig.show()

In [504]:
hist_df = comments_df.copy()
hist_df["color"] = np.select(
    [hist_df["sentiment"].eq(5), hist_df["sentiment"].eq(1)],
    ["green", "red"],
    "blue"
)

fig = px.histogram(hist_df, 
                   x="sentiment",
                   title='Distribution of Sentiment for Comments',text_auto=True,
                   color="color",
                   color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "blue": "blue"
                    }).update_layout(
                   xaxis_title="Sentiment", yaxis_title="Number of Comments", showlegend=False)
fig.show()

In [432]:
fig = px.box(posts_df, x="sentiment", y='post_relevance_score')
fig.show()

In [435]:
# fig = px.scatter(res_df, x="comment_relevance", y='comment_score', color='sentiment')
# fig.show()

In [436]:
fig = px.box(res_df, x="post_id", y='comment_relevance')
fig.show()

## Relevance over time